## **Parametric Models and Regressors** 

Please stay tuned, here we will have the parametric version of the models...

In [1]:
from robot_descriptions.ur5_description import URDF_PATH

from darli.parametric import Functional
from darli.backend import CasadiBackend
from darli.robots import manipulator
from darli.functional import FunctionalStateSpace

In [2]:
manip = manipulator(Functional, CasadiBackend, URDF_PATH)

# How to access base parameters (from URDF)


In [3]:
urdf_params = manip.backend.base_parameters()

# How to access dynamical properties

In [4]:
manip.inertia

Function(inertia:(q[6],parameters[60])->(inertia[6x6]) SXFunction)

In [5]:
manip.forward_dynamics

Function(forward_dynamics:(q[6],v[6],tau[6],parameters[60])->(dv[6]) SXFunction)

# How to access regressors

In [6]:
manip.regressors.torque

Function(torque_regressor:(q[6],v[6],dv[6])->(torque_regressor[6x60]) SXFunction)

In [7]:
manip.regressors.kinetic

Function(kinetic_regressor:(q[6],v[6])->(kinetic_regressor[1x60]) SXFunction)

In [8]:
manip.regressors.potential

Function(potential_regressor:(q[6])->(potential_regressor[1x60]) SXFunction)

# How to access the state space

In [9]:
state_space = FunctionalStateSpace(manip)

In [10]:
state_space.derivative

Function(derivative:(q[6],v[6],tau[6],theta[60])->(derivative[12]) SXFunction)